# Fantasy Football Schedule Generator
In this tutorial I will show you how to:
1. Get the teams from yahoo
2. Generate the schedule with various options
3. Validate the schedule
4. Upload the schedule (manual for now but might make a vid on automating it)


## Get the Schedule

In [108]:
all_teams=["Team "+str(x+1) for x in range(12)]
all_teams,len(all_teams)

(['Team 1',
  'Team 2',
  'Team 3',
  'Team 4',
  'Team 5',
  'Team 6',
  'Team 7',
  'Team 8',
  'Team 9',
  'Team 10',
  'Team 11',
  'Team 12'],
 12)

In [109]:
import random
random.shuffle(all_teams)#shuffles in place. Makes everything "random" -- technically pseudorandom but still should be good enough
all_teams

['Team 6',
 'Team 12',
 'Team 5',
 'Team 10',
 'Team 7',
 'Team 4',
 'Team 2',
 'Team 1',
 'Team 9',
 'Team 3',
 'Team 8',
 'Team 11']

To preserve the randomness you can set a "seed"  https://docs.python.org/3/library/random.html#random.seed but we just save the output later instead.

Since we randomized earlier we can use deterministic methods like %, the modulo operator, to simplify our code.

# Notes on the settings
Your divisions must be equally sized. And you must have an 1 or an even number of conferences

In [110]:
from collections import defaultdict#makes list append simpler

divisions=defaultdict(list)
for i,team in enumerate(all_teams):
    divisions[i//6].append(team)#4 teams per division
divisions

defaultdict(list,
            {0: ['Team 6', 'Team 12', 'Team 5', 'Team 10', 'Team 7', 'Team 4'],
             1: ['Team 2', 'Team 1', 'Team 9', 'Team 3', 'Team 8', 'Team 11']})

In [111]:

conferences=defaultdict(list)
conference_nums=[]
for division, teams in divisions.items():
    division_num=division//1
    conferences[division_num]+=teams
    conference_nums.append(division_num)
    
conferences,conference_nums


(defaultdict(list,
             {0: ['Team 6',
               'Team 12',
               'Team 5',
               'Team 10',
               'Team 7',
               'Team 4'],
              1: ['Team 2',
               'Team 1',
               'Team 9',
               'Team 3',
               'Team 8',
               'Team 11']}),
 [0, 1])

In [112]:

game_types={
    "divisional": 8,#Mandate a divisional game to start and stop the season - hard coded below
    "conference": 0,
    "non_conference": 4
}
game_types_per_week=[]
for k,v in game_types.items():
    game_types_per_week+=[k]*v
random.shuffle(game_types_per_week)
game_types_per_week.insert(0,"divisional")
game_types_per_week+=["divisional"]
game_types_per_week

['divisional',
 'divisional',
 'non_conference',
 'non_conference',
 'divisional',
 'divisional',
 'divisional',
 'divisional',
 'non_conference',
 'divisional',
 'non_conference',
 'divisional',
 'divisional',
 'divisional']

# Constraints 
- 1 game max between nondivisional teams. 
- 2 games between divisional

These constraints are relatively simple. Each constraint can be coded by a corresponding universe for each team. To simplify things we will not mix weeks, each week has a shared theme: divisional, non_conference, and conference. When a game is played we will remove that opponent from each teams' universe. 
  
If you wanted to add a "don't play a team 2 weeks in a row rule" you would need to add a short term dont_play set for each team to include all the invalid teams for that team, for that week. 

In [117]:

def gen_matchups(all_teams_, universe_):
    '''
    Attempts to generate a week of matchups using each teams universe ()
    '''
    universe_=copy.deepcopy(universe_)
    weekly_teams_playing = set()
    weekly_matchups = []
    teams_remaining=set(all_teams_)
    
    while teams_remaining:
        team_1=teams_remaining.pop()
        candidates=list(set(universe_[team_1])-weekly_teams_playing)
#         print(candidates)
        team_2=random.choice(candidates)
        universe_[team_1].remove(team_2)
        universe_[team_2].remove(team_1)

        weekly_teams_playing=weekly_teams_playing.union(set([team_1,team_2]))
        weekly_matchups.append([team_1,team_2])
        teams_remaining.remove(team_2)
#         print(weekly_matchups)
#         print()
    return weekly_matchups

{'Team 6': ['Team 5',
  'Team 4',
  'Team 12',
  'Team 7',
  'Team 10',
  'Team 5',
  'Team 4',
  'Team 12',
  'Team 7',
  'Team 10'],
 'Team 12': ['Team 5',
  'Team 6',
  'Team 4',
  'Team 7',
  'Team 10',
  'Team 5',
  'Team 6',
  'Team 4',
  'Team 7',
  'Team 10'],
 'Team 5': ['Team 6',
  'Team 4',
  'Team 12',
  'Team 7',
  'Team 10',
  'Team 6',
  'Team 4',
  'Team 12',
  'Team 7',
  'Team 10'],
 'Team 10': ['Team 5',
  'Team 6',
  'Team 4',
  'Team 12',
  'Team 7',
  'Team 5',
  'Team 6',
  'Team 4',
  'Team 12',
  'Team 7'],
 'Team 7': ['Team 6',
  'Team 4',
  'Team 12',
  'Team 5',
  'Team 10',
  'Team 6',
  'Team 4',
  'Team 12',
  'Team 5',
  'Team 10'],
 'Team 4': ['Team 5',
  'Team 6',
  'Team 12',
  'Team 7',
  'Team 10',
  'Team 5',
  'Team 6',
  'Team 12',
  'Team 7',
  'Team 10'],
 'Team 2': ['Team 1',
  'Team 9',
  'Team 3',
  'Team 8',
  'Team 11',
  'Team 1',
  'Team 9',
  'Team 3',
  'Team 8',
  'Team 11'],
 'Team 1': ['Team 9',
  'Team 3',
  'Team 8',
  'Team 2',
 

In [121]:
from itertools import combinations,permutations
import copy


divisional_opponent_universe={}
conference_opponent_universe={}
nonconference_opponent_universe={}

for division, teams in divisions.items():
    for team in teams:
        divisional_opponent_universe[team]=list(set(teams)-set([team]))*2

for conference, teams in conferences.items():
    for team in teams:
        conference_opponent_universe[team]=list(set(teams)-set([team])-set(divisional_opponent_universe[team]))
        #copy is helpful because of pointer vs primitive rules in Python. It may not be necessary in this case though
        nonconference_opponent_universe[team]=copy.deepcopy(conferences[(conference+1)%2])#with 2 conferences this maps 0 to 1 and 1 to 0

universes={
    "divisional": divisional_opponent_universe,
    "conference": conference_opponent_universe,
    "non_conference": nonconference_opponent_universe,
}

universes['divisional']

weekly_teams_playing=defaultdict(set)
games_played={}
weekly_matchups=[]


for week, game_type in enumerate(game_types_per_week):
    #print(week)
        
    print(week,game_type)
    matchups=None
    #Potentially invalid matchups can happen depending on the contstraints so each week will have a given number of attempts. 
    #It's also possible a prior week leads to an impossible state. We will only support rerunning on the week level
    ATTEMPTS=500
    for attempt in range(ATTEMPTS):
        try:
            matchups=gen_matchups(all_teams,universes[game_type])
            break
        except:
            continue
    if not matchups:
        raise Exception(f"No valid matchups found for week {week} after {ATTEMPTS} attempts. Please check your settings are valid or try increasing the number of attempts")
    else:
        for name, universe in universes.items():
            for team_1,team_2 in matchups:
                for a,b in permutations([team_1,team_2], 2):
                    if b in universe.get(a,[]):                        
                        universes[name][a].remove(b)

        weekly_matchups.append(matchups)

[x[0] for x in weekly_matchups]

0 divisional
1 divisional
2 non_conference
3 non_conference
4 divisional
5 divisional
6 divisional
7 divisional
8 non_conference
9 divisional
10 non_conference
11 divisional
12 divisional
13 divisional


[['Team 1', 'Team 11'],
 ['Team 1', 'Team 3'],
 ['Team 1', 'Team 5'],
 ['Team 1', 'Team 10'],
 ['Team 1', 'Team 2'],
 ['Team 1', 'Team 8'],
 ['Team 1', 'Team 8'],
 ['Team 1', 'Team 3'],
 ['Team 1', 'Team 4'],
 ['Team 1', 'Team 11'],
 ['Team 1', 'Team 6'],
 ['Team 1', 'Team 2'],
 ['Team 1', 'Team 9'],
 ['Team 1', 'Team 9']]

# Validating the Output
We will output the occurrences of each matchup.  
  
No row should have a value greater than 2. And we should see nice c

In [122]:
import numpy as np
N=len(all_teams)
occurences_matrix=np.zeros((N,N))

for weeks_matchups in weekly_matchups:
    
    for team_1,team_2 in weeks_matchups:
        for a,b in permutations([team_1,team_2], 2):
            occurences_matrix[all_teams.index(a)][all_teams.index(b)]+=1
occurences_matrix

array([[0., 2., 2., 2., 2., 2., 0., 1., 1., 0., 1., 1.],
       [2., 0., 2., 2., 2., 2., 1., 0., 0., 1., 1., 1.],
       [2., 2., 0., 2., 2., 2., 1., 1., 1., 1., 0., 0.],
       [2., 2., 2., 0., 2., 2., 1., 1., 1., 1., 0., 0.],
       [2., 2., 2., 2., 0., 2., 1., 0., 0., 1., 1., 1.],
       [2., 2., 2., 2., 2., 0., 0., 1., 1., 0., 1., 1.],
       [0., 1., 1., 1., 1., 0., 0., 2., 2., 2., 2., 2.],
       [1., 0., 1., 1., 0., 1., 2., 0., 2., 2., 2., 2.],
       [1., 0., 1., 1., 0., 1., 2., 2., 0., 2., 2., 2.],
       [0., 1., 1., 1., 1., 0., 2., 2., 2., 0., 2., 2.],
       [1., 1., 0., 0., 1., 1., 2., 2., 2., 2., 0., 2.],
       [1., 1., 0., 0., 1., 1., 2., 2., 2., 2., 2., 0.]])

# Input into Yahoo

In [123]:

import pandas as pd

matchups_df=pd.DataFrame(weekly_matchups).T
matchups_df.columns=[x+1 for x in matchups_df.columns]
matchups_df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,"[Team 1, Team 11]","[Team 1, Team 3]","[Team 1, Team 5]","[Team 1, Team 10]","[Team 1, Team 2]","[Team 1, Team 8]","[Team 1, Team 8]","[Team 1, Team 3]","[Team 1, Team 4]","[Team 1, Team 11]","[Team 1, Team 6]","[Team 1, Team 2]","[Team 1, Team 9]","[Team 1, Team 9]"
1,"[Team 6, Team 4]","[Team 6, Team 7]","[Team 6, Team 11]","[Team 6, Team 9]","[Team 6, Team 12]","[Team 6, Team 4]","[Team 6, Team 7]","[Team 6, Team 12]","[Team 6, Team 8]","[Team 6, Team 10]","[Team 9, Team 5]","[Team 6, Team 10]","[Team 6, Team 5]","[Team 6, Team 5]"
2,"[Team 9, Team 8]","[Team 9, Team 2]","[Team 9, Team 4]","[Team 7, Team 11]","[Team 9, Team 11]","[Team 9, Team 2]","[Team 9, Team 11]","[Team 9, Team 8]","[Team 9, Team 10]","[Team 9, Team 3]","[Team 7, Team 8]","[Team 9, Team 3]","[Team 7, Team 4]","[Team 7, Team 4]"
3,"[Team 7, Team 10]","[Team 4, Team 10]","[Team 7, Team 3]","[Team 4, Team 8]","[Team 7, Team 5]","[Team 7, Team 12]","[Team 4, Team 12]","[Team 7, Team 10]","[Team 7, Team 2]","[Team 7, Team 12]","[Team 4, Team 11]","[Team 7, Team 5]","[Team 12, Team 10]","[Team 12, Team 10]"
4,"[Team 12, Team 5]","[Team 12, Team 5]","[Team 12, Team 8]","[Team 12, Team 3]","[Team 4, Team 10]","[Team 3, Team 11]","[Team 3, Team 2]","[Team 4, Team 5]","[Team 12, Team 11]","[Team 4, Team 5]","[Team 12, Team 2]","[Team 4, Team 12]","[Team 3, Team 8]","[Team 3, Team 11]"
5,"[Team 3, Team 2]","[Team 8, Team 11]","[Team 2, Team 10]","[Team 2, Team 5]","[Team 3, Team 8]","[Team 5, Team 10]","[Team 5, Team 10]","[Team 2, Team 11]","[Team 3, Team 5]","[Team 2, Team 8]","[Team 3, Team 10]","[Team 8, Team 11]","[Team 2, Team 11]","[Team 2, Team 8]"


In [124]:
import pandas as pd
#pd.DataFrame(weekly_matchups).to_csv("matchups.csv")#save for posterity

In [125]:
divisions

defaultdict(list,
            {0: ['Team 6', 'Team 12', 'Team 5', 'Team 10', 'Team 7', 'Team 4'],
             1: ['Team 2', 'Team 1', 'Team 9', 'Team 3', 'Team 8', 'Team 11']})

In [126]:
WEEK_NUM=1
print(game_types_per_week[WEEK_NUM-1])
weekly_matchups[WEEK_NUM-1]

divisional


[['Team 1', 'Team 11'],
 ['Team 6', 'Team 4'],
 ['Team 9', 'Team 8'],
 ['Team 7', 'Team 10'],
 ['Team 12', 'Team 5'],
 ['Team 3', 'Team 2']]

# Additional Information
Optimization theory and constraint programming deal with scheduling problem. It can be very, very complex. 
- [How the NFL may deal with it](https://www.math.cmu.edu/~af1p/Teaching/OR2/Projects/P56/OR-Final-Paper.pdf)
- [Constraint Programming wikipedia page](https://en.wikipedia.org/wiki/Constraint_programming#:~:text=Constraint%20programming%20(CP)%20is%20a,a%20set%20of%20decision%20variables.)
- [Coursera Course](https://www.coursera.org/learn/discrete-optimization) -- I took this, kinda hard I think I flunked by the end but did learn some useful stuff.